# Colab Depth Visualizer for Multiple EXR Files

Upload multiple `.exr` files and visualize their R, G, B channels and combined RGB in Google Colab.

In [ ]:
# Install system and Python dependencies
!apt-get update -qq
!apt-get install -y libopenexr-dev
!pip install -q OpenEXR Imath numpy matplotlib

In [ ]:
%matplotlib inline
import OpenEXR
import Imath
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import os

In [ ]:
# Upload multiple EXR files
print("Select and upload one or more .exr files:")
uploaded = files.upload()
filepaths = list(uploaded.keys())
print(f"Uploaded {len(filepaths)} files:")
for fp in filepaths:
    print(" -", fp)

In [ ]:
# Define helper functions
def load_exr(filename):
    exr = OpenEXR.InputFile(filename)
    dw = exr.header()['dataWindow']
    W = dw.max.x - dw.min.x + 1
    H = dw.max.y - dw.min.y + 1
    pt = Imath.PixelType(Imath.PixelType.FLOAT)
    channels = {}
    for c in ('R', 'G', 'B'):
        buf = exr.channel(c, pt)
        arr = np.frombuffer(buf, dtype=np.float32)
        channels[c] = arr.reshape(H, W)
    return channels

def normalize_channel(arr):
    mn, mx = arr.min(), arr.max()
    return (arr - mn) / (mx - mn + 1e-8)

In [ ]:
# Process and visualize each uploaded file
for filepath in filepaths:
    print(f"\nProcessing {filepath}")
    channels = load_exr(filepath)
    for c in channels:
        print(f"  {c} channel shape: {channels[c].shape}")
    normed = {c: normalize_channel(channels[c]) for c in channels}
    rgb = np.stack([normed[c] for c in ('R','G','B')], axis=-1)

    fig, axs = plt.subplots(1, 4, figsize=(20, 5))
    for i, c in enumerate(('R','G','B')):
        im = axs[i].imshow(channels[c], cmap='plasma')
        axs[i].set_title(f'{c} Channel')
        axs[i].axis('off')
        plt.colorbar(im, ax=axs[i], fraction=0.046, pad=0.04)
    axs[3].imshow(rgb)
    axs[3].set_title('Combined RGB')
    axs[3].axis('off')
    plt.suptitle(filepath, fontsize=14)
    plt.tight_layout()
    plt.show()